*Dataset Preprocessing*

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\klkum\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\klkum\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
file_path = 'data/COCO_overlaping_dataset.txt'

data = []
with open(file_path, 'r') as f:
    for line in f:
        parts = line.strip().split(',')
        if len(parts) >= 3:
          keywords, descriptions, target_keywords = parts[0], parts[1], parts[2]
          data.append((keywords, descriptions, target_keywords))
  
df = pd.DataFrame(data, columns=['keywords', 'descriptions', 'target_keywords'])
df.head()

,keywords,descriptions,target_keywords
0,laptop carton comicbook,a laptop that has stickers on its cover is sit...,{'laptop'}
1,snorkel ski tennis ball,two frames of a woman in the air on a tennis c...,{'tennis'}
2,sorrel hog barrel,a brown horse eating from a hallowed out metal...,{'barrel'}
3,ballplayer baseball footballhelmet,a man throwing a baseball from a mound on a field,{'baseball'}
4,ballplayer baseball football helmet,a boys baseball game with a batter catcher and...,{'baseball'}


In [ ]:
print(f"Total number of records: {len(df)}")

Total number of records: 70797


In [9]:
def preprocess_text(text):
  text = re.sub(r"[^a-zA-Z]", " ", text)
  text = text.lower()
  tokens = word_tokenize(text)
  tokens = [word for word in tokens if word not in stopwords.words("english")]
  return " ".join(tokens)

df['cleaned_descriptions'] = df['descriptions'].apply(preprocess_text)
print("Descriptions cleaned successfully!")
df[['descriptions', 'cleaned_descriptions']].head()

Descriptions cleaned successfully!


,descriptions,cleaned_descriptions
0,a laptop that has stickers on its cover is sit...,laptop stickers cover sitting table
1,two frames of a woman in the air on a tennis c...,two frames woman air tennis court
2,a brown horse eating from a hallowed out metal...,brown horse eating hallowed metal barrel
3,a man throwing a baseball from a mound on a field,man throwing baseball mound field
4,a boys baseball game with a batter catcher and...,boys baseball game batter catcher umpire
